<a href="https://colab.research.google.com/github/xvanbeurden/Individual_assignment/blob/main/Pong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
!python -m atari_py.import_roms gdrive/MyDrive/ROMS

copying mr_do.bin from gdrive/MyDrive/ROMS/Mr. Do! (1983) (CBS Electronics, Ed English) (4L4478) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/mr_do.bin
copying lost_luggage.bin from gdrive/MyDrive/ROMS/Lost Luggage (Airport Mayhem) (1982) (Apollo - Games by Apollo, Larry Minor, Ernie Runyon, Ed Salvo) (AP-2004) [no opening scene] ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/lost_luggage.bin
copying elevator_action.bin from gdrive/MyDrive/ROMS/Elevator Action (1983) (Atari, Dan Hitchens) (CX26126) (Prototype) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/elevator_action.bin
copying asterix.bin from gdrive/MyDrive/ROMS/Asterix (AKA Taz) (1984) (Atari, Jerome Domurat, Steve Woita) (CX2696).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/asterix.bin
copying riverraid.bin from gdrive/MyDrive/ROMS/River Raid (1982) (Activision, Carol Shaw) (AX-020, AX-020-04) ~.bin to /usr/local/lib/python3.7/dist-packa

In [ ]:
import gym
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
# def create_pong_env(): 
#   return gym.make("Pong-v0", frameskip=5)
# env = create_pong_env()
# env.seed(1); # for reproducibility

In [4]:
# Atari's Pong RL 

import numpy as np
import pickle
import gym
import cv2
import pandas as pd
import math


# hyperparameters
H = 200 # number of hidden layer neurons
batch_size = 10 # every how many episodes to do a param update?
learning_rate = 1e-4
gamma = 0.85 # discount factor for reward
decay_rate = 0.99 # decay factor 
resume = True # resume from previous checkpoint?

# model initialization
D = 80 * 80 # is input dimensionality
path = '/content/gdrive/MyDrive/Model/'
#save.p = lr = 1e-4, gamma = 0.99, policy is stoch
#save2.p = lr = 1e-4, gamma = 0.99, policy is decaying eps_greedy
#save3.p = lr = 1e-4, gamma = 0.85, policy is decaying eps_greedy
#save4.p = lr = 1e-4, gamma = 0.3, policy is decaying eps_greedy
#save5.p = lr = 1e-3, gamma = 0.85, policy is decaying eps_greedy
#save6.p = lr = 1e-5, gamma = 0.85, policy is decaying eps_greedy
#save7.p = lr = 1e-4, gamma = 0.85, policy is decaying eps_greedy, H = 400

if resume:
    model= pickle.load(open(path + 'save3.p', 'rb'))
else:
    model= {}
    model['W1']= np.random.randn(H,D) / np.sqrt(D) 
    model['W2']= np.random.randn(H) / np.sqrt(H)
  
grad_buffer = { k : np.zeros_like(v) for k,v in model.items() } # update buffers that add up gradients over a batch
rmsprop_cache = { k : np.zeros_like(v) for k,v in model.items() } # rmsprop memory

def sigmoid(x): 
    return 1.0 / (1.0 + np.exp(-x)) # sigmoid squashes the logprob between 0 and 1

def prepro(img):
  ## remove unnecessary pixels and downsample image and convert to gray-scale
    img = img[35:195] 
    img = img[::2,::2,0] 
    img[img == 144] = 0 
    img[img == 109] = 0 
    img[img != 0] = 1 

    # #also reduce the image by 50% to speed up training
    # scale_percent = 50 # percent of original size
    # width = int(I.shape[1] * scale_percent / 100)
    # height = int(I.shape[0] * scale_percent / 100)
    # dim = (width, height)
      
    # # resize image
    # I = cv2.resize(I, dim, interpolation = cv2.INTER_AREA)
    return img.astype(np.float).ravel()

def discount_rewards(r):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(0, r.size)):
      if r[t] != 0: running_add = 0 # When the 
      running_add = running_add * gamma + r[t]
      discounted_r[t] = running_add
    return discounted_r

def policy_forward(x):
    h = np.dot(model['W1'], x)
    h[h<0] = 0 # ReLU nonlinearity
    logp = np.dot(model['W2'], h)
    p = sigmoid(logp)
    return p, h # returns probability of moving up and the hidden state

def policy_backward(episodeh, epdlogp):
    dW2 = np.dot(episodeh.T, epdlogp).ravel()
    dh = np.outer(epdlogp, model['W2'])
    dh[episodeh <= 0] = 0 # backpro prelu
    dW1 = np.dot(dh.T, epx)
    # print('dW1 =', dW1)
    # print('dW2 =', dW2)
    return {'W1':dW1, 'W2':dW2}

df = pd.DataFrame(columns = ('episode','total_reward','running_mean'))
env = gym.make("Pong-v0", frameskip = 5)
observation = env.reset()
prev_state = None # used in computing the difference frame
xs = []
hs = []
dlogps = []
drs = []
running_reward = None
reward_sum = 0
episode_number = 0
eps_start = 0.9
eps_end = 0.05
eps_decay = 100
batch_number = 0
while True:
    # preprocess the observation, set input to network to be difference image
    cur_state = prepro(observation)
    state = cur_state - prev_state if prev_state is not None else np.zeros(D)
    prev_state = cur_state

    # forward the policy network and sample an action from the returned probability
    aprob, h = policy_forward(state)
    eps_threshold = eps_end + (eps_start - eps_end) * math.exp(-1 * batch_number / eps_decay)
    if np.random.uniform() > eps_threshold:
      action = 2 if aprob > 0.5 else 3
    else:
      action = 2 if np.random.uniform() < 0.5 else 3 # 50% chance of moving up/down


    # record various intermediates (needed later for backprop)
    xs.append(state) # observation
    hs.append(h) # hidden state
    y = 1 if action == 2 else 0 
    dlogps.append(y - aprob) # grad that encourages the action

    # step the environment and get new measurements
    observation, reward, done, info = env.step(action)
    reward_sum += reward

    drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

    if done: # an episode finished
      episode_number += 1

      # stack together all inputs, hidden states, action gradients, and rewards for this episode
      epx = np.vstack(xs) #episode state
      episodeh = np.vstack(hs) #episode hidden state
      epdlogp = np.vstack(dlogps) #episode delta log probability
      episoderew = np.vstack(drs) #episode reward
      # memory reset
      xs = []
      hs = []
      dlogps = []
      drs = [] 

      # compute the discounted reward backwards through time
      discounted_episoderew = discount_rewards(episoderew) #discounted episode reward
      # print('discounted_episoderew =', discounted_episoderew)
      # normalize the rewards to be unit normal (helps control the gradient estimator variance)
      discounted_episoderew -= np.mean(discounted_episoderew)
      discounted_episoderew /= np.std(discounted_episoderew)

      epdlogp *= discounted_episoderew # modulate the gradient with advantage (PG magic happens right here.)
      grad = policy_backward(episodeh, epdlogp)
      for k in model: grad_buffer[k] += grad[k] # accumulate grad over batch

      # perform rmsprop parameter update every batch_size episodes
      if episode_number % batch_size == 0:
        batch_number += 1
        for k,v in model.items():
          g = grad_buffer[k] # gradient
          rmsprop_cache[k] = decay_rate * rmsprop_cache[k] + (1 - decay_rate) * g**2
          model[k] += learning_rate * g / (np.sqrt(rmsprop_cache[k]) + 1e-5)
          grad_buffer[k] = np.zeros_like(v) # reset batch gradient buffer

      # boring book-keeping
      if running_reward is None:
        running_reward = reward_sum 
      else:
        running_reward * 0.99 + reward_sum * 0.01
      print('episode = %f, total reward = %f. running mean= %f' % (episode_number, reward_sum, running_reward))
      df.loc[df.shape[0]] = [episode_number, reward_sum, running_reward]
      df.to_csv('gdrive/MyDrive/Pong_results_colab3_7.xlsx')
      if episode_number % 100 == 0: 
        pickle.dump(model, open(path + 'save3.p', 'wb'))
      reward_sum = 0
      observation = env.reset() # reset env
      prev_state = None


episode = 1.000000, total reward = -21.000000. running mean= -21.000000
episode = 2.000000, total reward = -20.000000. running mean= -21.000000
episode = 3.000000, total reward = -19.000000. running mean= -21.000000
episode = 4.000000, total reward = -21.000000. running mean= -21.000000
episode = 5.000000, total reward = -21.000000. running mean= -21.000000


KeyboardInterrupt: ignored